In [2]:
import torch.nn.functional as F  # functions of the neural network library
import load_dataset as load  # module with function to load data
from Conv_Layer import *
from Pooling import *
import gc
from Dense import *

In [3]:
I = 28
O = I//2
n = 2*I
k = 2
K = I//4

device = torch.device("mps")
batch_size = 10  # the number of examples per batch
train_loader, test_loader, dim_in, dim_out = load.load_MNIST(batch_size=batch_size)
scala = 1000
reduced_loader = reduce_MNIST_dataset(train_loader, scala)
# list_gates = [(i, j) for i in range(O-1) for j in range(i+1, O-1)]
# list_gates = [(i, j) for i in range(O-1) for j in range(O-1)]
list_gates1 = [(i, i+1) for i in range(O//2-1)]
list_gates2 = [(O//2-1-i, O//2-2-i) for i in range(O//2-1)]
# [(13,12),(12,11),...,(1,0)] O//2 = 14

In [4]:
full_model = nn.Sequential(Conv_RBS_density_I2(I,K,device), Pooling_2D_density(I,O,device),
                           Conv_RBS_density_I2(I//2,K,device), Pooling_2D_density(I//2,O//2,device),
                           Basis_Change_I_to_HW_density(O//2, device), Dense_RBS_density(O//2, list_gates1, device), Dense_RBS_density(O//2, list_gates2, device))

In [5]:
loss_function = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.Adagrad(full_model.parameters(), lr=1e-1, lr_decay=1e-6, weight_decay=0, initial_accumulator_value=1e-6, eps=1e-10)

def train_net(network, train_loader, loss_function, optimizer, device):
    network.train()
    train_loss = 0
    train_accuracy = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        # batch_data tensor size: batch_size * 784
        # init_density_matrix tensor size: batch_size * 784 * 784
        init_density_matrix = to_density_matrix(F.normalize(data.squeeze().resize(data.size()[0],784), p=2, dim=1).to(device), device)
        out_network = network(init_density_matrix) # out tensor size: batch * 91 * 91

        # training
        targets = get_batch_projectors(target, batch_size, int(binom(O,2)), device)
        loss = loss_function(out_network,targets)
        train_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # predict digital number
        predict_number_vector = get_predict_number_vector(out_network, device)
        predict_number = torch.argmax(predict_number_vector, dim=1).to(device)
        acc = predict_number.eq(target.to(device).view_as(predict_number).to(device)).sum().item()
        train_accuracy += acc

        # delete variable to free memory
        del out_network
        gc.collect()

    train_accuracy /= len(train_loader.dataset)
    train_loss /= (batch_idx + 1)
    return train_accuracy, train_loss
    

## one direction full connect dense layer

In [7]:
for epoch in range(10):
    train_accuracy, train_loss = train_net(full_model, reduced_loader, loss_function, optimizer, device)
    print(f'Epoch {epoch}: Loss = {train_loss:.6f}, accuracy = {train_accuracy*100:.4f} %')

/Users/letao/anaconda3/envs/env0/lib/python3.11/site-packages/torch/_tensor.py:885: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


Epoch 0: Loss = 1.201768, accuracy = 6.6667 %
Epoch 1: Loss = 1.125369, accuracy = 18.3333 %
Epoch 2: Loss = 1.112012, accuracy = 21.6667 %
Epoch 3: Loss = 1.105667, accuracy = 25.0000 %
Epoch 4: Loss = 1.099544, accuracy = 28.3333 %
Epoch 5: Loss = 1.098427, accuracy = 25.0000 %
Epoch 6: Loss = 1.094153, accuracy = 26.6667 %
Epoch 7: Loss = 1.091470, accuracy = 25.0000 %
Epoch 8: Loss = 1.088402, accuracy = 25.0000 %
Epoch 9: Loss = 1.087548, accuracy = 30.0000 %


## two directions full connect dense layer

In [11]:
for epoch in range(10):
    train_accuracy, train_loss = train_net(full_model, reduced_loader, loss_function, optimizer, device)
    print(f'Epoch {epoch}: Loss = {train_loss:.6f}, accuracy = {train_accuracy*100:.4f} %')

Epoch 0: Loss = 1.083454, accuracy = 8.3333 %
Epoch 1: Loss = 1.066504, accuracy = 13.3333 %
Epoch 2: Loss = 1.060517, accuracy = 21.6667 %
Epoch 3: Loss = 1.056199, accuracy = 21.6667 %
Epoch 4: Loss = 1.051593, accuracy = 18.3333 %
Epoch 5: Loss = 1.051199, accuracy = 25.0000 %
Epoch 6: Loss = 1.050228, accuracy = 20.0000 %
Epoch 7: Loss = 1.048553, accuracy = 28.3333 %
Epoch 8: Loss = 1.046531, accuracy = 30.0000 %
Epoch 9: Loss = 1.046679, accuracy = 26.6667 %


## one direction connect dense layer

In [15]:
for epoch in range(10):
    train_accuracy, train_loss = train_net(full_model, reduced_loader, loss_function, optimizer, device)
    print(f'Epoch {epoch}: Loss = {train_loss:.6f}, accuracy = {train_accuracy*100:.4f} %')

Epoch 0: Loss = 1.195591, accuracy = 13.3333 %
Epoch 1: Loss = 1.136768, accuracy = 20.0000 %
Epoch 2: Loss = 1.126574, accuracy = 18.3333 %
Epoch 3: Loss = 1.119312, accuracy = 20.0000 %
Epoch 4: Loss = 1.113273, accuracy = 23.3333 %
Epoch 5: Loss = 1.114937, accuracy = 23.3333 %
Epoch 6: Loss = 1.111925, accuracy = 25.0000 %
Epoch 7: Loss = 1.107330, accuracy = 31.6667 %
Epoch 8: Loss = 1.108248, accuracy = 23.3333 %
Epoch 9: Loss = 1.105261, accuracy = 26.6667 %


## two directions connect dense layer

In [6]:
for epoch in range(10):
    train_accuracy, train_loss = train_net(full_model, reduced_loader, loss_function, optimizer, device)
    print(f'Epoch {epoch}: Loss = {train_loss:.6f}, accuracy = {train_accuracy*100:.4f} %')

/Users/letao/anaconda3/envs/env0/lib/python3.11/site-packages/torch/_tensor.py:885: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


Epoch 0: Loss = 1.186986, accuracy = 13.3333 %
Epoch 1: Loss = 1.151447, accuracy = 18.3333 %
Epoch 2: Loss = 1.134318, accuracy = 30.0000 %
Epoch 3: Loss = 1.129754, accuracy = 26.6667 %
Epoch 4: Loss = 1.116979, accuracy = 26.6667 %
Epoch 5: Loss = 1.108586, accuracy = 28.3333 %
Epoch 6: Loss = 1.103779, accuracy = 28.3333 %
Epoch 7: Loss = 1.101172, accuracy = 25.0000 %
Epoch 8: Loss = 1.100642, accuracy = 28.3333 %
Epoch 9: Loss = 1.095588, accuracy = 28.3333 %
